In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

Using TensorFlow backend.
c:\users\skarkosz\appdata\local\continuum\anaconda3\envs\snakes\lib\site-packages\tensorflow\python\framework\dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\skarkosz\appdata\local\continuum\anaconda3\envs\snakes\lib\site-packages\tensorflow\python\framework\dtypes.py:472: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\skarkosz\appdata\local\continuum\anaconda3\envs\snakes\lib\site-packages\tensorflow\python\framework\dtypes.py:473: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 =

In [2]:

with open('dataset.txt','r',encoding='utf-8') as txt:
    text = txt.read()
print('corpus length:', len(text))

corpus length: 114788


In [3]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 98


In [4]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 38250


In [5]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [6]:
y.shape

(38250, 98)

In [7]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

Build model...


In [9]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [14]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=20,
          callbacks=[print_callback])

Epoch 1/20
38144/38250 [============================>.] - ETA: 0s - loss: 2.3819
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "j drogi.

14 stycznia 1896



XII 

Kto "
j drogi.

14 stycznia 1896



XII 

Kto tak w szale w szczęcze się w tak w sobie w szczę w tak pokorzy w się szarczy przed w sływa podybne w tak postaczy pomierne od marczy do w szczym i soko w się szkale się w tak jak się szkalany w słycha postały w szczędzie pomielny postysz w się pokole w się strzed w szczym postały w tak w się się postać w się w nie ziemie pokory w się zamieć podnie zamienie od moczy pomieczy się się od mach podyszc
----- diversity: 0.5
----- Generating with seed: "j drogi.

14 stycznia 1896



XII 

Kto "
j drogi.

14 stycznia 1896



XII 

Kto tylko tak przez boz nie się w wypostać miecza,
To miat zamiecza pomit skamień w szczychy w spocerce będym w koli ziepia dzie się pok przebły,
jakno ciał zamyście pięki rozposławy sale na zamiernych strawała ręko od gr

będoni znago
w tróciś podobutnych dziejmień —
Głobędz płonię much co odnewo
  będą grozną niebow zawiot
Zamilnyć i mopodny.

A to sumiepię poproką
kwiat nzetówy jak to lubi niebo dziela,,
I każdy koń nos go dych the stele na sścieże oganić i topiaj, ę wczorti strorszki będzie chmuki i porostr.
nie meminości z chcić,
Gdy nadziejety smątej człowiękile;
o I wyszczerum? jedna przywieli będą
mich w iwi
----- diversity: 1.2
----- Generating with seed: "gmat



ktoś kiedyś
zobaczył ślad stopy
"
gmat



ktoś kiedyś
zobaczył ślad stopy
puk, a korąc wobszystych czaty,
ą 1śwódząst chma torch? na dajrcoków, anjabyci po środ wzlysu y.
! wednogo wej jubia tua kłaską
gdzić — co wyk notje, w głąsicie przebłoniów.
  I mieczcim rózka miałem
tygóry lutjem
Anarzie ciebugał dalekiż na …mieniłe —
Murając
tygo zasudonną peobiękach łokou na trzczęłnych
zarona san śwogę w światłu wszędni.
Omabu gwłize.  dujonłoV,
świdzicię mgjąciejniów
i tesą r
38250/38250 [==============================] - 31s 812us/step - lo

Na Termopilach ja się nie wieko,
Więc nie wiedząc w czarne czyste się zamieni,
któr jeszcze marteni słowa zamienić spomieni
nie wierzące czas i pozmarłej,
Więc wielkie zakleli twałem pod szukami tych strzele,
I wierzcze z szary, co szkla ziemi,
kartym w nich wieków słowa zamienić, ziemię zmierzał i zaszczenia za szczemi,
I w krychach powiedzie w ciemność,
Więc nie wodzielnych powiedzie zamieni,
któr jeszcze wybra spomnieni
----- diversity: 0.5
----- Generating with seed: "zy — żywych.

Na Termopilach ja się nie "
zy — żywych.

Na Termopilach ja się nie zduręci;
Przez zduszął wyśli stał się wstrzęda twoich,
powiecz z tak w groba, wyszy, które za aniecia drzewa i drżące —
i sych wieczących powiedzie
i wiekrone, z miłość burzy zaszernych na ziemi,
Po szara złote zymotka gorzy.

Ze ścierwane krzykie zamienia drogie,
jak w przeszał zatrzeje jak drogie w błędzie,
Z kragu na obraci za niewiekiącej się z zamitenie bodzą w krzybie skrzydych czas namieni,
----- diversity: 1.0
----- Generating wi

c:\users\skarkosz\appdata\local\continuum\anaconda3\envs\snakes\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  


orzeko i smutnie wielkie się przestrzecie.




Adam Asnyk

Mollona



Na przywalczenie powierz słachemnie,
I że zielen nad drogiesz w na ziemi,
I serce przewieki w zapornieniu i strach
I szekam porosków porobi
----- diversity: 0.5
----- Generating with seed: "ci i konie
nad rozlewanym ogniem, a on c"
ci i konie
nad rozlewanym ogniem, a on cośń których podniewszych
I żądnące przed głód zmierzie w wielkim podnienie.




Adam Asnyk

Dyszlist powietnie
pod niesię potrzeba i samotnych na siebi.




Adam Asnyk

Oh Aśnt
to trzewa patrzem się w brzegu i czarne,
teraz gdzie zamarzy w zaparzem się strach
I nie spod dorowisko dzieli nie zdała.

I przechodzi nasze się w porodowno zniewieka,
Po stów wzasta w niesiednej ręki.




Adam Asnyk

Jen 
----- diversity: 1.0
----- Generating with seed: "ci i konie
nad rozlewanym ogniem, a on c"
ci i konie
nad rozlewanym ogniem, a on cos zdartwą

o lot :
satone of meglina d ślżgbim owidą, która mglna w mół.
Nie pędnie umioni i z skiem,
ą przed błzcowy w stał

Posędzi wypanzy miastym narodów,
ocpannie emkie zatrżałym w biały,
I długcy dzicia mi do pakażać
----- diversity: 1.2
----- Generating with seed: "a rzeźbę w nim. Jak w skrzepłą krew
zanu"
a rzeźbę w nim. Jak w skrzepłą krew
zanurów, żi białe,
To to AlniePszy sarg długi twat otmarty.
wyanumI w bomię prymatermów było
jeślo oluba consku z pokuta.
Szłowi z pokolenie kopysku
Przeżazł przednia, którem jak an ona chcerą.

Jest ofi smueskościa te mnieJ!
Zu światmo mrok, nieśmi łak
konaków kiudnych pieśbie?
na gimory odziwiarają ongod skijącem,
a to zmytalość ieczozy, że chcej rywawy, i zbroć —
niona przeuę ziemię,

tak ośpysi mu
38250/38250 [==============================] - 31s 810us/step - loss: 1.4065
Epoch 13/20
38144/38250 [============================>.] - ETA: 0s - loss: 1.3745
----- Generating text after Epoch: 12
----- diversity: 0.2
----- Generating with seed: "e roztrząsać ciekawa,
Ten zgrzyt w harmo"
e roztrząsać ciekawa,
Ten zgrzyt w harmoże,
nic przez większała zamienia.




Ada

Przostalej te się prowidziskie powiedzących w niebardy płomienie
na pieści spukają w gromie,
I wielkie pomięka w sobie, gdy podniewe,
które z postałem, co wielkiem stałe w pusty,
W cerzące gnią sw
----- diversity: 0.5
----- Generating with seed: ", głosu rozpoznania,
i jeśliś nie przypo"
, głosu rozpoznania,
i jeśliś nie przyporą spojegowne głosą złote sczysteko wieka,
I ścichnej miacha się wszyka



Nie mięcią popokać gdybne,
I wiatrm zatrwały się w poczym się w nocy.

I 
Juk go szczęśli, podróżnym przychany,
Więc w cieniu w niewarwają gdybniego spoi moteni,
I mierzędzo, co dusze twoł posąga,
abu wysiąp przyszałem, że ziemi donów te sczyszy,
A terago zdała zawsze zatrżała niebie,
wielkie za małkie wspomarzeni wyłańsu s
----- diversity: 1.0
----- Generating with seed: ", głosu rozpoznania,
i jeśliś nie przypo"
, głosu rozpoznania,
i jeśliś nie przypownine, bednim zmrokutenie wirzychach dziejszeko bezwielisząc,
ciele, nie zdonazy, co drogie
deszczej dą, co szczęśli krewa,
Teryc się świę

cza świat okopach dośli za obrazku.
A nie idzie winiony uści, marłość z bół.
no
----- diversity: 1.2
----- Generating with seed: "serce twoje chwytać ich nie umie.

Tyś ł"
serce twoje chwytać ich nie umie.

Tyś łębi kuMdońkę i mu jak smucu,
śmierciach rocz, no za obłętem, czerk aniości sułok
  bratnych, króce wybówzm kakuty
gwiazdy i mieczki skruty, i które w son zytuśnie — prac?
pownie zasleżen up strac.
zławość cieśni na pałmożny.

M w crajen w skaziersią pogiłą
skonmia się pogdniutie wniej
Wysamy w mu, bez skierue
pocinów 1źwienne har dłona.

Czepotem pośrzek chodzi pogię ilony z nagie
spadnąna dragiec
38250/38250 [==============================] - 31s 814us/step - loss: 1.2608
Epoch 20/20
38144/38250 [============================>.] - ETA: 0s - loss: 1.2390
----- Generating text after Epoch: 19
----- diversity: 0.2
----- Generating with seed: "w, dalekie o czyjąś śmierć.
Płyną przeze"
w, dalekie o czyjąś śmierć.
Płyną przeze skazienna obłosów przestąk i słobiecie w boles
    nie ma

In [15]:

neural_texts = list()
fan_fic_len = 400

for n in range(10):
    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.7]:

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence

        for i in range(fan_fic_len):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
    neural_texts.append(generated)

c:\users\skarkosz\appdata\local\continuum\anaconda3\envs\snakes\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  


In [16]:
neural_texts

['\no! przyświećcie blaskiem skrzydeł swoich wziele;\nTa przybo moc i sków słuchach\nTu świerowa słunech dziej\nCieha się wszystkie liść\njak mi jud wieków płynie przyadli zgiśliech spiarą jak dziernące wysólida,\nA śmierć nigu nas wygo bosem słoszeczie.\n\nNiece z ucholna.\n\nNie było chłoskie ust wielką ze spała,\nNiebeju na wieć poniosłym wiedną się bając,\nprzyświ tak w mu sarzał się czarną okragitem,\nnie brzmierzały w uchrów,\nNo jeze samnonych j',
 ' przyczyny,\nSwych bliźnich zawsze sądzim potrzebie.\n\nNiech kurtonia przywatem?\n\nNiech żypają ziemiał śmiersą w krygnienie wraje,\nscerziwe znawiste, i trzech ci się wskija znajdzie,\nnie będzie obrozne, zdybrago,\nZemsnąc poczy wydobne drogi.\n\ntyn głaz wytazi się w chcerust lude brzy\nto oto, ledzą zmieniali i tak w sobie\n\n\n\nI\n\n\n\nZiebie imy strzele —\nniu nie jedność który w nim zielią.\n\nNie drzenie spracował ludzki poznonia\n\n\n\nO nowy, co ro',
 'ę w wnętrznościach twych duch struchlałych —\nNiech potraba,\nBy w ś